# ✨ Chapter 10: Visual FX, Lighting & Particles 

Physics makes a game *work*. Visuals make a game *feel alive*. ✨

In this chapter, we add the "WOW" factor. We will build:
1.  **Particle System**: Explosions and smoke. 💥
2.  **Screen Shake**: Impact feedback. 🫨
3.  **Lighting Engine**: Using Blend Modes for glow effects. 💡

## 1. The Particle Class

A particle is just a simple object that:
1.  Has a position (x, y)
2.  Has a velocity (vx, vy)
3.  Has a purely visual lifespan (it fades out or shrinks).

In [ ]:
import pygame
import random

class Particle:
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.color = color
        
        # Random velocity for explosion effect
        self.vx = random.uniform(-2, 2)
        self.vy = random.uniform(-2, 2)
        
        # Lifespan: How long until it dies?
        self.timer = random.randint(20, 40)
        self.size = random.randint(4, 8)

    def update(self):
        self.x += self.vx
        self.y += self.vy
        self.timer -= 1
        # Shrink over time
        if self.size > 0:
            self.size -= 0.1

    def draw(self, screen):
        if self.size > 0:
            pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), int(self.size))

## 2. The Particle Manager

Just like we have an `all_sprites` group, we want a list to manage thousands of particles efficiently.

In [ ]:
class ParticleSystem:
    def __init__(self):
        self.particles = []

    def emit(self, x, y, color, count=10):
        # Create 'count' particles at position (x,y)
        for _ in range(count):
            self.particles.append(Particle(x, y, color))

    def update(self):
        # Update all particles
        for p in self.particles:
            p.update()
            
        # Remove dead particles (list comprehension is fast!)
        self.particles = [p for p in self.particles if p.timer > 0]

    def draw(self, screen):
        for p in self.particles:
            p.draw(screen)

## 3. Lighting with Blend Modes 🕯️

Pygame supports **Add** and **Multiply** blending. 

- **BLEND_ADD**: Adds colors together. Great for fire, lasers, and magical glow. (Black becomes transparent).
- **BLEND_MULT**: Multiplies colors. Great for shadows and day/night cycles. (White becomes transparent).

### Creating a "Glow" Function

In [ ]:
def draw_glow(screen, x, y, radius, color):
    # 1. Create a temporary surface
    glow_surf = pygame.Surface((radius * 2, radius * 2))
    
    # 3. Handle transparency (ColorKey)
    glow_surf.set_colorkey((0, 0, 0)) # Black is "Clear"
    
    # 4. Draw the circle on the temp surface
    # We draw a few concentric circles for a soft gradient
    pygame.draw.circle(glow_surf, (color[0]//4, color[1]//4, color[2]//4), (radius, radius), radius)
    pygame.draw.circle(glow_surf, (color[0]//2, color[1]//2, color[2]//2), (radius, radius), radius * 0.7)
    pygame.draw.circle(glow_surf, color, (radius, radius), radius * 0.4)
    
    # 5. BLIT using BLEND_ADD
    # This adds the light values to the existing screen pixels!
    screen.blit(glow_surf, (x - radius, y - radius), special_flags=pygame.BLEND_ADD)

## 4. Screen Shake 🫨

Screen shake gives weight to impacts. It's surprisingly easy to implement!

Instead of drawing everything at `(0, 0)`, we draw the world at a slightly offset position `(offset_x, offset_y)` that vibrates randomly.

In [ ]:
shake_timer = 0
render_offset = [0, 0]

def apply_shake():
    global shake_timer, render_offset
    if shake_timer > 0:
        render_offset[0] = random.randint(-5, 5)
        render_offset[1] = random.randint(-5, 5)
        shake_timer -= 1
    else:
        render_offset = [0, 0]
        
# In the Game Loop:
# if player_hit_enemy:
#     shake_timer = 20  # Shake for 20 frames!
# 
# apply_shake()
# screen.blit(background, (render_offset[0], render_offset[1]))  # Everything shifts!

## 🛠️ Challenge: The Neon Night

1.  Fill the background with a very dark Blue `(10, 10, 30)`.
2.  Draw your player.
3.  Use the `draw_glow` function to add a blue glow around the player, and red glows around enemies.
4.  Notice how the colors add up and make it look like a neon city!

**Cyberpunk vibes achieved.** 🕶️